# Estatística Geral Teoria e Aplicações
## Pós Graduação em Ciência de Dados e Big Data - PUC Minas

Datasets:
* https://www.kaggle.com/PromptCloudHQ/imdb-data

Referências:
* http://www.scipy-lectures.org/packages/statistics/index.html

Dicas:
* https://github.com/JosPolfliet/pandas-profiling

### Índice:
* [Tabelas de frequências](#Resumo-de-dados-com-tabelas-de-frequências)
* [Medidas de Tendência Central](#Medidas-de-tendência-central)
    * [Média](#Média)
    * [Mediana](#Mediana)
    * [Moda](#Moda)
* [Medidas de dispersão](#Medidas-de-dispersão)
    * [Amplitude](#Amplitude)
    * [Desvio padrão](#Desvio-padrão)
* [Medidas de posição](#Medidas-de-posição)
    * [Separatrizes](#Separatrizes)
    * [Score z](#Score-z)
* [Correlação](#Correlação)
* [Representações gráficas](#Representações-gráficas)
    * [Gráfico de linhas](#Gráfico-de-linhas)
    * [Gráfico de barras](#Gráfico-de-barras)
    * [Gráfico de setores](#Gráfico-de-setores)
    * [Histograma](#Histograma)
    * [Diagrama de dispersão](#Diagrama-de-dispersão)
    * [Boxplot](#Boxplot)
* [Probabilidades](#Probabilidades)
    * [Simulações](#Simulações)
    * [O problema de Monty Hall](#O-problema-de-Monty-Hall)
* [Inferência](#Inferência)
    * [Intervalos de confiança](#Intervalos-de-confiança)
    * [Testes de hipóteses](#Testes-de-hipóteses)

In [ ]:
import math
import random

import matplotlib.pyplot as plt
from matplotlib import style
import pandas as pd
import scipy
import scipy.stats
import numpy as np

%matplotlib inline

In [ ]:
imdb_data = pd.read_csv('dados/IMDB-Movie-Data.csv')

In [ ]:
imdb_data.head()

In [ ]:
imdb_data.info()

In [ ]:
imdb_data['main_genre'] = imdb_data.Genre.apply(lambda x: x.split(',')[0]).values
imdb_data.head()

In [ ]:
imdb_data.Genre.unique()

## Resumo de dados com tabelas de frequências

### Dados discretos: Gênero

In [ ]:
# O método value_counts de um pandas DataFrame ou pandas Series retorna os valores e a frequência
#  de ocorrência dos mesmos ordenado do mais frequente para o menos frequente

imdb_data.main_genre.value_counts()

In [ ]:
# Para a frequência relativa é só usar o parâmetro normalize=True

imdb_data.main_genre.value_counts(normalize=True)

In [ ]:
# Pandas possui a função cumsum() para fazer a soma cumulativa
# Com isso podemos fazer a Frequência acumulada

imdb_data.main_genre.value_counts().cumsum()

In [ ]:
# Gerando um DataFrame com todas essas frequências
pd.DataFrame({'Fa': imdb_data.main_genre.value_counts(),
              'Fac': imdb_data.main_genre.value_counts().cumsum(),
              'Fr': imdb_data.main_genre.value_counts(normalize=True)})

### Dados contínuos: Rating

In [ ]:
n = imdb_data.shape[0]

In [ ]:
# Para dados contínuos precisamos definir os limites de classes ou, ao menos, a quantidade de classes

# Calculando apenas a quantidade de classes através da raiz de n
# Usamos o sort=False para que os dados fiquem ordenados pela classe e não pela frequência
# Passamos também o parâmetro bins com a quantidade de classes
imdb_data.Rating.value_counts(sort=False, bins=math.sqrt(n))

In [ ]:
# Uma outra forma mais "bonitinha" é usar a função pd.cut
# 
# Esta função identifica em qual bin está o dado. Porém ela retorna uma pd.Series com os bins 
# onde os dados estão. Temos, então, que fazer o value_counts.

pd.cut(imdb_data.Rating, bins=math.sqrt(n)).value_counts(sort=False)

In [ ]:
# Outra vantagem do pd.cut é que podemos passar os limites das classes e não apenas o intervalo de classes.

# Vamos calcular "na unha" as classes para a coluna Rating
k = math.sqrt(n)
k = 20 if k > 20 else int(k) # Limita a quantidade de classes em 20. Opcional
h = (imdb_data.Rating.max() - imdb_data.Rating.min()) / (k - 1)
bins = [imdb_data.Rating.min() - h/2]
for i in range(k):
    bins.append(bins[i] + h)
pd.cut(imdb_data.Rating, bins).value_counts(sort=False)

In [ ]:
imdb_data[imdb_data.Rating < 2]

In [ ]:
imdb_data[imdb_data.Rating == imdb_data.Rating.max()]

In [ ]:
# A função pd.cut é útil para transformar uma variável contínua em discreta no seu DataFrame

imdb_data['Rating Bin'] = pd.cut(imdb_data.Rating, bins)
imdb_data.head()

[Voltar](#Índice:)

## Medidas de tendência central

### Média

In [ ]:
# pandas DataFrames e pandas Series têm métodos específicos para calcular a média das colunas

imdb_data.mean()

In [ ]:
# Se, por algum motivo, quiser calcular a média das linhas

imdb_data.mean(axis=1)

In [ ]:
# Se não for um pandas DataFrame, precisa da biblioteca Numpy para alta performance
# ou do módulo statistics sem precisar instalar nada

import numpy as np
import statistics

data_list_ímpar = [1, 2, 3, 4, 5]
data_list_par = [1, 2, 3, 4]

print(np.mean(data_list_ímpar))
print(np.mean(data_list_par))
print(statistics.mean(data_list_ímpar))

In [ ]:
# Cuidado com os missings. 
statistics.mean(imdb_data.Metascore)

In [ ]:
# numpy.mean já resiste bem aos missings
np.mean(imdb_data.Metascore)

In [ ]:
# Gerando dados para comparar a performance
big_data = [random.gauss(0, 1) for i in range(1000000)]

In [ ]:
%%timeit
np.mean(big_data)

In [ ]:
%%timeit
statistics.mean(big_data)

### Propriedades algébricas da média

In [ ]:
big_média = np.mean(big_data)
desvios = [data - big_média for data in big_data]
print("Soma dos desvios em relação à média: {:0.8f}".format(sum(desvios)))

In [ ]:
constantes = [random.gauss(0, 0.1) for i in range(10)]

for constante in constantes:
    print("Soma dos quadrados dos desvios em relação a {:0.3f}: {:0.8f}".format(constante,
                                                                                sum([(data - constante)**2 for data in big_data])))
    
print("Soma dos quadrados dos desvios em relação à média: {:0.8f}".format(constante,
                                                                            sum([(data - big_média)**2 for data in big_data])))

In [ ]:
print("Média dos dados:", big_média)
média3 = np.mean([np.multiply(data, 3) for data in big_data])
print("Média dos dados multiplicados por três:", média3)
print("Diferença entre essa média e big_média x 3:", média3 - (big_média * 3.0))
print('Essa média é "igual" a big_média x 3?', math.isclose(média3, big_média*3))
print()
média2 = np.mean([data/2 for data in big_data])
print("Média dos dados divididos por dois:", média2)
print("Diferença entre essa média e big_média / 2:", média2 - big_média/2)
print('Essa média é "igual" a big_média / 2?', math.isclose(média2, big_média/2))

### Mediana

In [ ]:
# Mesma coisa com a mediana
imdb_data.median()

In [ ]:
print(np.median(data_list_ímpar))
print(np.median(data_list_par))
print(statistics.median(data_list_par))

In [ ]:
# Cuidado com dados faltantes

np.median(imdb_data.Metascore)

In [ ]:
# Pra isso numpy tem uma série de métodos que começam com numpy.nan
np.nanmedian(imdb_data.Metascore)

In [ ]:
# Como a média e a mediana se comportam com dados extremos?
data = [2, 3, 4, 2, 3, 1, 3]
print("Média de {}: {:0.2f}".format(data, np.mean(data)))
print("Mediana de {}: {}".format(data, np.median(data)))
data_extremo = data + [1000]
print("Média de {}: {}:".format(data_extremo, np.mean(data_extremo)))
print("Mediana de {}: {}".format(data_extremo, np.median(data_extremo)))

### Moda

In [ ]:
imdb_data.mode()

In [ ]:
# Numpy não tem um método para cálculo da moda. Esse método está no módulo scipy.stats
from scipy import stats
moda = stats.mode(['a', 'b', 'c', 'd', 'e', 'e', 'f', 'f', 'f', 'g', 'h'])
print("Moda: {[0]}".format(moda))
print("Frequência: {[1]}".format(moda))

In [ ]:
statistics.mode(['a', 'b', 'c', 'd', 'e', 'e', 'f', 'f', 'f', 'g', 'h'])

In [ ]:
# O problema do método mode do módulo statistics é que se existir mais de uma moda
# o código gera uma exceção


statistics.mode(['a', 'b', 'c', 'd', 'e', 'e', 'f', 'f', 'g', 'g', 'h'])

In [ ]:
# Numpy/Scipy retornam a primeira moda, não retornam todas 😒
stats.mode(['a', 'b', 'c', 'd', 'e', 'e', 'f', 'f', 'g', 'g', 'h'])

In [ ]:
%%timeit
statistics.mode([1, 2, 3, 3, 4, 4, 4, 4, 5, 5, 5, 6, 7, 7, 8])

In [ ]:
%%timeit
stats.mode([1, 2, 3, 3, 4, 4, 4, 4, 5, 5, 5, 6, 7, 7, 8])

## Medidas de dispersão
### Amplitude

In [ ]:
# pandas não nos fornece um método para calcular a amplitude...

imdb_data[['Rank', 'Year', 'Runtime (Minutes)', 'Rating', 'Votes',
       'Revenue (Millions)', 'Metascore']].apply(lambda col: col.max() - col.min())

In [ ]:
# numpy.ptp não funcionou diretamente com o DataFrame. Tive que transformar em um array

# Novamente o problema com os dados faltantes
np.ptp(imdb_data[['Rank', 'Year', 'Runtime (Minutes)', 'Rating', 'Votes', 'Revenue (Millions)', 'Metascore']].values, axis=0)


In [ ]:
# Ou então...
imdb_data[['Rank', 'Year', 'Runtime (Minutes)', 'Rating', 'Votes', 'Revenue (Millions)', 'Metascore']].apply(np.ptp)

### Desvio padrão

In [ ]:
# Aqui aparece a questão do ddof. O denominador da fórmula do
# desvio padrão é n-ddof. Como na fórmula que definimos o
# denominador é n-1 recomendo usar ddof=1; mas só para efeitos
# acadêmicos. Para efeitos práticos em um projeto de DS pode
# deixar ddof=0 pois o desvio padrão nesses casos é um orientador
# e não um número sobre o qual efetivamente se tomarão decisões
# erradas com base em diferenças na segunda ou terceira casa decimal

imdb_data.std(ddof=1)

In [ ]:
np.std(imdb_data.Metascore, ddof=1)

In [ ]:
np.std([1, 2, 3, 4], ddof=1)

In [ ]:
k = [1, 2, 3, 4]
np.sqrt(sum([(i-2.5)**2 for i in k])/3)

## Medidas de posição
### Separatrizes

In [ ]:
# No método quantile do pandas DataFrame ou pandas Series o percentil vai de 0 a 1
imdb_data.quantile(0.25)

In [ ]:
# No numpy.percentile ou numpy.nanpercentile já é de 0 a 100
np.nanpercentile(imdb_data['Revenue (Millions)'], 25)

In [ ]:
data_list_par

In [ ]:
np.percentile(data_list_par, 75)

In [ ]:
# Comparando o manual com o automático

#Usando a interpolação 'midpoint' para comparar com o que nós convencionamos
np.percentile(data_list_par, 75, interpolation='midpoint')

In [ ]:
# Fazendo 'na unha'


def percentil(data, i):
    n = len(data)
    # Econtrando a posição
    P = i * (n + 1) / 100
    if P == int(P):
        percent = data[P-1]
    else:
        P = int(P)
        percent = (data[P-1] + data[P])/2

    return percent

percentil(data_list_par, 75)

In [ ]:
percentil(data_list_ímpar, 75)

In [ ]:
np.percentile(data_list_ímpar, 75)

In [ ]:
data_list_ímpar

### _Score_ z

In [ ]:
# O pandas não tem uma função pronta mas é relativamente fácil fazer a transformação
imdb_data[['Runtime (Minutes)', 'Rating', 'Votes',
       'Revenue (Millions)', 'Metascore']].apply(lambda col: (col - col.mean())/col.std())

In [ ]:
# Numpy também não, então tem que apelar para o scipy.stats
# O problema é que ele não consegue tratar as colunas com missing

print(stats.zscore(imdb_data[['Runtime (Minutes)', 'Rating', 'Votes',
       'Revenue (Millions)', 'Metascore']], ddof=1))

In [ ]:
# Neste caso a melhor opção é usar o scikit-learn pois ele armazena
# a média e o desvio padrão usados na transformação

# O 'problema' é que não aceita valores missing....

from sklearn import preprocessing

# Cria a instância do objeto StandardScaler
zscorer = preprocessing.StandardScaler()

# "Armazena" os dados de média e desvio padrão das colunas
zscorer.fit(imdb_data[['Runtime (Minutes)', 'Rating', 'Votes']])

# Efetivamente aplica a transformação utilizando a média e desvio armazenados
# Você pode aplicar o método transform em bases futuras com as mesmas variáveis!
zscorer.transform(imdb_data[['Runtime (Minutes)', 'Rating', 'Votes']])

## Correlação

In [ ]:
imdb_data[['Year', 'Runtime (Minutes)', 'Rating', 'Votes', 'Revenue (Millions)', 'Metascore']].corr()

## Representações gráficas
### Gráfico de linhas

In [ ]:
style.use('ggplot')
# Como os dados da base MetaScore não são contínuos pelo tempo não podemos (ou pelo menos não devemos) fazer gráficos de linhas
# Portanto vou gerar dados aleatórios
random_data = pd.Series([random.gauss(mu=50, sigma=3) for r in range(30)])
# E colocar um índice com datas de trinta dias atrás até ontem
random_data.index = pd.date_range(pd.Timestamp.now().date() - pd.Timedelta('30D'), periods=30)

ax = random_data.plot(figsize=(12, 5), marker='.')
ax.set_title("Vendas em milhões de reais")

for x, y in zip(random_data.index, random_data.values):
    ax.annotate('{:0.2f}'.format(y), xy=(x, y), ha='center', va='bottom', rotation=45)

### Gráfico de barras

In [ ]:
imdb_data['Year'].value_counts(sort=False).plot.bar(title='Quantidade de filmes lançados por ano', figsize=(10, 5));

### Gráfico de setores

In [ ]:
# Aqui damos um "tapa" no visual. Alteramos o mapa de cores.
# Para ver os mapas de cores visite https://matplotlib.org/examples/color/colormaps_reference.html

imdb_data.Year.value_counts().plot.pie(figsize=(5,5), autopct='%0.1f%%', colormap='Spectral');

### Histograma

In [ ]:
imdb_data[['Metascore', 'Rating', 'Revenue (Millions)', 'Runtime (Minutes)']].hist(figsize=(12, 5));

In [ ]:
# Fazendo "na unha"
# Já fizemos, lembram????

# Vamos calcular "na unha" as classes para a coluna Rating
n = len(imdb_data.Rating.values)
k = math.sqrt(n)
k = 20 if k > 20 else int(k) # Limita a quantidade de classes em 20. Opcional
h = (imdb_data.Rating.max() - imdb_data.Rating.min()) / (k - 1)
bins = [imdb_data.Rating.min() - h/2]
for i in range(k):
    bins.append(bins[i] + h)
ax = pd.cut(imdb_data.Rating, bins).value_counts(sort=False).plot.bar(width=1)
ax.set_title('Rating');

### Diagrama de dispersão

In [ ]:
pd.tools.plotting.scatter_matrix(imdb_data[['Metascore', 'Rating', 'Revenue (Millions)', 'Runtime (Minutes)']], diagonal='kde');

In [ ]:
imdb_data.plot.scatter('Rating', 'Metascore');

### Boxplot

In [ ]:
imdb_data[['Metascore', 'Rating', 'Revenue (Millions)', 'Runtime (Minutes)']].boxplot();

## Probabilidades

### Simulações

In [ ]:
import random# Probabilidade de nascerem mais que 127 homens em 152 nascimentos

mais_que_127 = 0
testes = 10000
for i in range(testes):
    nascimentos = [random.choice([0, 1]) for i in range(152)]
    if sum(nascimentos) >= 127:
        mais_que_127 += 1
print("Probabilidade de nascerem mais que 127 homens em 152 nascimentos: {:0.2%}".format(mais_que_127 / testes))

In [ ]:
# Probabilidade de que duas pessoas em um grupo de 25 tenham nascido no mesmo dia

datas_iguais = 0
testes = 100000
for i in range(testes):
    nascimentos = [random.randint(1, 365) for i in range(25)]
    if len(nascimentos) > len(set(nascimentos)):
        datas_iguais += 1
print("Probabilidade de que duas pessoas em um grupo de 25 tenham a mesma data de nascimento: {:0.2%}".format(datas_iguais / testes))

In [ ]:
# Probabilidade de termos uma sequência de seis "caras" ou seis "coroas" em 200 lançamentos de moeda

tem_sequência = 0
testes = 10000
for i in range(testes):
    lançamentos = "".join([str(random.choice([0, 1])) for i in range(200)])
    if 6*"0" in lançamentos or 6*"1" in lançamentos:
        tem_sequência += 1

print('Probabilidade de termos uma sequência de seis "caras" ou seis "coroas" em 200 lançamentos de moeda: {:0.2%}'.format(tem_sequência / testes)) 

### O problema de Monty Hall

In [ ]:
# Sempre trocando de porta
venceu = 0
testes = 1000 

for i in range(testes):
    portas = {1, 2, 3}
    porta_premiada = random.randint(1, 3)
    porta_escolhida = random.randint(1, 3)
    porta_aberta = portas.difference([porta_premiada, porta_escolhida]).pop()
    porta_escolhida = portas.difference([porta_escolhida, porta_aberta]).pop()
    if porta_premiada == porta_escolhida:
        venceu += 1
        
print("Probabilidade de vitória sempre trocando de porta: {:0.2%}".format(venceu / testes))

In [ ]:
# Nunca trocando de porta
venceu = 0
testes = 1000

for i in range(testes):
    portas = {1, 2, 3}
    porta_premiada = random.randint(1, 3)
    porta_escolhida = random.randint(1, 3)
    if porta_premiada == porta_escolhida:
        venceu += 1
        
print("Probabilidade de vitória nunca trocando de porta: {:0.2%}".format(venceu / testes))

In [ ]:
# Trocando eventualmente de porta
venceu = 0
testes = 1000

for i in range(testes):
    portas = {1, 2, 3}
    porta_premiada = random.randint(1, 3)
    porta_escolhida = random.randint(1, 3)
    porta_aberta = portas.difference([porta_premiada, porta_escolhida]).pop()
    if random.randint(0, 1):
        porta_escolhida = portas.difference([porta_escolhida, porta_aberta]).pop()
    if porta_premiada == porta_escolhida:
        venceu += 1
        
print("Probabilidade de vitória trocando eventualmente de porta: {:0.2%}".format(venceu / testes))

## Inferência

### Intervalos de confiança

In [ ]:
# Fazendo "na unha"
alpha = 0.05
z = scipy.stats.norm().ppf(1 - alpha/2)
data = [random.gauss(50, 0.2) for i in range(50)]
n = len(data)
média = np.mean(data)
desvio = np.std(data, ddof=1)
E = z*desvio/np.sqrt(n)
print("Intervalo de confiança para a média dos dados: ({}, {}, {})".format(média-E, média, média+E))

In [ ]:
# Usando scipy
scipy.stats.norm.interval(0.95, loc=média, scale=desvio/np.sqrt(n))

# Por que usar o desvio dividido pela raiz de n?
# https://stackoverflow.com/questions/28242593/correct-way-to-obtain-confidence-interval-with-scipy/28243282#28243282

In [ ]:
# Calcular o intervalo de confiança para a média do MetaScore
scipy.stats.norm.interval(0.95, imdb_data.Metascore.mean(), imdb_data.Metascore.std(ddof=1)/np.sqrt(len(imdb_data.Metascore)))

In [ ]:
# Usando a distribuição T de student.
# Funciona bem para amostras pequenas SE OS DADOS SEGUEM UMA DISTRIBUIÇÃO NORMAL.

# Fazendo "na unha"
alpha = 0.05
t = scipy.stats.t.ppf(1 - alpha/2, len(data)-1)
data = [random.gauss(300, 2) for i in range(10)]
n = len(data)
média = np.mean(data)
desvio = np.std(data, ddof=1)
E = t*desvio/np.sqrt(n)
print("Intervalo de confiança para a média dos dados: ({}, {}, {})".format(média-E, média, média+E))

In [ ]:
scipy.stats.t.interval(0.95, len(data)-1, np.mean(data), np.std(data, ddof=1)/np.sqrt(n))

### Testes de hipóteses

In [ ]:
imdb_data.groupby('Year')['Metascore'].mean().plot.bar();

In [ ]:
# Será que os filmes de 2006 e 2007 têm notas médias melhores que os filmes dos outros anos?
até_2007 = imdb_data.loc[imdb_data.Year <= 2007, 'Metascore']
pós_2007 = imdb_data.loc[imdb_data.Year > 2007, 'Metascore']
scipy.stats.ttest_ind(até_2007, pós_2007, nan_policy='omit')

In [ ]:
# Comparando a diferença ano a ano através de uma ANOVA
#scipy.stats.f_oneway()

ar = imdb_data.groupby('Year').apply(lambda x: x.Metascore[x.Metascore.notnull()].reset_index(drop=True)).unstack(level=0).values
arT = imdb_data.groupby('Year').apply(lambda x: x.Metascore[x.Metascore.notnull()].reset_index(drop=True)).unstack(level=0).T.values

In [ ]:
df = imdb_data.groupby('Year').apply(lambda x: x.Metascore[x.Metascore.notnull()].reset_index(drop=True)).unstack(level=0)
data = [df[col].dropna().values for col in df]
df

In [ ]:
scipy.stats.f_oneway(*data)

In [ ]:
plt.figure()
plt.boxplot(data);

In [ ]:
plt.figure()
plt.boxplot([até_2007.dropna().values, pós_2007.dropna().values]);